In [41]:
!pip install findspark pyspark

Looking in indexes: https://edge.artifactory.ouroath.com:4443/artifactory/api/pypi/pypi-mirror/simple

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [42]:
import os
import findspark
import pyspark

In [43]:
spark_home = os.environ.get('SPARK_HOME', '/opt/homebrew/Cellar/apache-spark/3.5.4/libexec')

In [44]:
findspark.init()

from pyspark.sql import SparkSession

In [45]:
spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

In [46]:
spark

# Answer 1

v3.5.4


In [47]:
data_url = "/Users/ajain17/Documents/Developer/de-zoomcamp-homework-2025/hw5/data/yellow_tripdata_2024-10.parquet"
df = spark.read.parquet(data_url)
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [48]:
ans2_df = df.repartition(4).write.mode("overwrite").parquet("./hw5/answers/ans2-output/")

In [50]:
!ls -ltrgh ./hw5/answers/ans2-output/

total 194880
-rw-r--r--@ 1 staff    24M Feb 26 20:35 part-00000-3ff9eed7-d6d1-4060-bcf1-056bb4ed3498-c000.snappy.parquet
-rw-r--r--@ 1 staff    24M Feb 26 20:35 part-00001-3ff9eed7-d6d1-4060-bcf1-056bb4ed3498-c000.snappy.parquet
-rw-r--r--@ 1 staff    24M Feb 26 20:35 part-00002-3ff9eed7-d6d1-4060-bcf1-056bb4ed3498-c000.snappy.parquet
-rw-r--r--@ 1 staff    24M Feb 26 20:35 part-00003-3ff9eed7-d6d1-4060-bcf1-056bb4ed3498-c000.snappy.parquet
-rw-r--r--@ 1 staff     0B Feb 26 20:35 _SUCCESS


# Answer 2

25MB
